Test and compare a model that only saw wind set (WS) 2 and 3 during training. Test only on WS3.
Each gas source position was simulated with one of the three wind sets, i.e., the model is only trained with 2/3 of the gas source positions.

In [1]:
# ~~~~~~~~~~~~~~~~
# PyTorch Model

import torch
import yaml
from models.decoder import architectures

with open("models/decoder/decoder_params.yaml") as file:
    params = yaml.safe_load(file)

decoder = architectures.LightningDecoderNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
decoder.load_state_dict(torch.load("models/decoder/decoder.pth"))
decoder.eval();

decoder_light = architectures.LightningDecoderNet(params["inner_dims"], params["seq_len"], params["learning_rate"])
decoder_light.load_state_dict(torch.load("models/decoder/decoder_lightTraining.pth"))
decoder_light.eval();

# ~~~~~~~~~~~~~~~~
# Kernel DM+V

from models.kernel_dmv.my_kernel_dmv import KernelDMV
kdm = KernelDMV()

# ~~~~~~~~~~~~~~~~
# GMRF

from models.gmrf.my_gmrf import myGMRF
gmrf = myGMRF()

In [2]:
from torch.utils import data
from data.gdm_dataset import GasDataSet

dataset = GasDataSet("data/30x25_reducedPositions/test_onlyWS1.pt")

In [3]:
import gdm_metrics

rmse = {"decoder": 0,
        "decoder_light": 0, 
        "gmrf": 0,
        "kdm": 0}

kld = { "decoder": 0,
        "decoder_light": 0,
        "gmrf": 0,
        "kdm": 0}

In [4]:
from tqdm import tqdm

list_of_models = [
                  "decoder",
                  "decoder_light",
                    "gmrf",
                    "kdm",
                 ]

dataloader = iter(data.DataLoader(dataset, shuffle=False, drop_last=True))

for X, y in tqdm(dataloader):
    with torch.no_grad(): 
        if "decoder" in list_of_models:
            y_decoder = decoder(X.squeeze(1))
            rmse["decoder"] += gdm_metrics.rmse(y_decoder, y)
            kld["decoder"] += gdm_metrics.kld(y_decoder, y)
        
        if "decoder_light" in list_of_models:
            y_decoder_light = decoder_light(X.squeeze(1))
            rmse["decoder_light"] += gdm_metrics.rmse(y_decoder_light, y)
            kld["decoder_light"] += gdm_metrics.kld(y_decoder_light, y)
            
        if "gmrf" in list_of_models:
            y_gmrf = gmrf.calculate(y.squeeze())[None,None,:]  # add two empty dimensions to be consistent with y.shape
            rmse["gmrf"] += gdm_metrics.rmse(y_gmrf, y)
            kld["gmrf"] += gdm_metrics.kld(y_gmrf, y)
            
        if "kdm" in list_of_models:
            y_kdm = kdm.calculate(y.squeeze())[None,None,:]    # add two empty dimensions to be consistent with y.shape
            rmse["kdm"] += gdm_metrics.rmse(y_kdm, y)
            kld["kdm"] += gdm_metrics.kld(y_kdm, y)

100%|█████████████████████████████████████| 10800/10800 [08:24<00:00, 21.42it/s]


In [5]:
# Print results
print(f"########\n# RMSE #\n########")
for elem in rmse:
    print(f"{elem}: \t {rmse[elem]/len(dataloader)}")
    
print(f"########\n# KL D #\n########")
for elem in kld:
    print(f"{elem}: \t {kld[elem]/len(dataloader)}")

########
# RMSE #
########
decoder: 	 0.06872546672821045
decoder_light: 	 0.0815834030508995
gmrf: 	 0.24751980237169982
kdm: 	 0.24200097596722714
########
# KL D #
########
decoder: 	 0.003116118023172021
decoder_light: 	 0.004383143503218889
gmrf: 	 0.030179070865121782
kdm: 	 0.029926305993333607


In [5]:
import numpy as np

rmses = [0.06872546672821045, 0.0815834030508995, 0.24751980237169982, 0.24200097596722714]

for r in rmses:
    print(r*len(dataset)/np.sqrt(len(dataset)))

7.142160008828695
8.478395948311661
25.723012415231494
25.149479151389535
